<a href="https://colab.research.google.com/github/AdamPeetz/stable_diffusion/blob/main/latent_space_walks_V4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow keras_cv --upgrade --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.6/721.6 kB 43.1 MB/s eta 0:00:00


In [ ]:
import keras_cv
from tensorflow import keras
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import math
from PIL import Image
from tqdm import tqdm
import os, shutil
from google.colab import drive
drive.mount('/content/gdrive')
from matplotlib import pyplot
import gc

Mounted at /content/gdrive


In [ ]:
tf.keras.backend.clear_session()

In [ ]:
# Enable mixed precision
# (only do this if you have a recent NVIDIA GPU)
keras.mixed_precision.set_global_policy("mixed_float16")

# define image dimensions
height = 768
width = 512

# Instantiate the Stable Diffusion model
model = keras_cv.models.StableDiffusion(img_height=height, img_width=width, jit_compile=True)

By using this model checkpoint, you acknowledge that its usage is subject to the terms of the CreativeML Open RAIL-M license at https://raw.githubusercontent.com/CompVis/stable-diffusion/main/LICENSE


# Helper Functions

In [ ]:
def save_images(image_library, image_count):
  for idx in tqdm(range(len(image_library))):
   plt.figure(figsize=(10,10))
   plt.imshow(image_library[idx][0])
   plt.axis('off')
   filename = '/content/gdrive/My Drive/planegan/sample_output_2/generated_plot_%05d.png' % (image_count+1)
   plt.savefig(filename, transparent=True,bbox_inches="tight",pad_inches=0.0 )
   plt.close()
   image_count += 1
  return image_count

In [ ]:
# function for spherical interpolation pathway
def slerp(t, v0, v1, DOT_THRESHOLD=0.9995):

    #convert V0 and V1 to numpy arrays
    v0 = v0.numpy()
    v1 = v1.numpy()

    #calculate dot product of vectors
    dot = np.sum(v0 * v1 / (np.linalg.norm(v0) * np.linalg.norm(v1)))
    #if transformation would not be meaningful, do linear interpolation
    if np.abs(dot) > DOT_THRESHOLD:
        v2 = (1 - t) * v0 + t * v1
    # otherwise, spheical interpolate
    else:
        theta_0 = np.arccos(dot)
        sin_theta_0 = np.sin(theta_0)
        theta_t = theta_0 * t
        sin_theta_t = np.sin(theta_t)
        s0 = np.sin(theta_0 - theta_t) / sin_theta_0
        s1 = sin_theta_t / sin_theta_0
        v2 = s0 * v0 + s1 * v1

        #convert vectors back to tensor object
        v2 = tf.convert_to_tensor(v2, float)

    #return new vector
    return v2

# SLERP Latent Space Interpolations Between Two Prompts

In [ ]:
# set noise seed for repeatability
seed = 44

# calculate noise input dimensions based on image dimensions
noise = tf.random.normal((768 // 8, 512// 8, 4), seed=seed)

# define positive string prompts
prompt_1 = "galaxies in space from a distance, fractal, psytrance, high quality, masterpeice, high definition, highly detailed, elegant, sharp focus, digital painting, scifi, fantasy, center frame"
prompt_2 = " devouring black holes in space, fractal, psytrance, high quality, masterpeice, high definition, highly detailed, elegant, sharp focus,concept art, digital painting, scifi, fantasy, center frame"
prompt_3 = "battle between heaven and hell over a planet in space, explosions, fractal, psytrance, high quality, masterpeice, high definition, highly detailed, sharp focus, digital painting, scifi, fantasy"
#prompt_4 = "crystal planet from orbit with a sun in the background, fractal, psytrance, high quality, masterpeice, high definition, highly detailed, elegant, sharp focus,concept art, character concepts, digital painting, scifi, fantasy, center frame, anatomically correct"
#prompt_5 = "crystal jungle, plants made of metal, fractal, psytrance, high quality, masterpeice, high definition, highly detailed, elegant, sharp focus,concept art, character concepts, digital painting, scifi, fantasy, center frame, anatomically correct"

# define negative string prompts
negative = "extra limbs, bad art, watermark, face, dull, pencils, error, malformed, low detail, jpeg artifacts, cropped, plain background, ugly, low-res, poorly drawn face, out of frame, poorly drawn hands, blurry, bad art, extra hands, bad anatomy, amputee, missing limbs, amputated"
# set number of interpolation steps between points in space
interpolation_steps = 256
# define model batch size (1 for 1 image at each step, if seed is fixed, will produce duplicates of the same image)
batch_size = 1
# create number of batches
batches = (interpolation_steps) // batch_size

# encode prompt to latent space
encoding_1 = tf.squeeze(model.encode_text(prompt_1))
encoding_2 = tf.squeeze(model.encode_text(prompt_2))
encoding_3 = tf.squeeze(model.encode_text(prompt_3))
#encoding_4 = tf.squeeze(model.encode_text(prompt_4))
#encoding_5 = tf.squeeze(model.encode_text(prompt_5))

# employ slerp
# divide points between 0 and 1 in linerspace based on the number of steps you want
tensor_list = []
for _, t in enumerate(np.linspace(0, 1, interpolation_steps)):
    new_latent = slerp(float(t), encoding_1, encoding_2)
    tensor_list.append(new_latent)

images = []
print(range(interpolation_steps))
for batch in range(interpolation_steps):
    print(batch)
    images.append(
      model.generate_image(
            tensor_list[batch],
            negative_prompt=negative,
            batch_size=batch_size,
            diffusion_noise=noise,
                           )
                  )

In [ ]:
counter = 0
save_images(images, counter)

100%|██████████| 256/256 [01:41<00:00,  2.51it/s]


256

In [ ]:
# employ slerp
# divide points between 0 and 1 in linerspace based on the number of steps you want
tensor_list = []
for _, t in enumerate(np.linspace(0, 1, interpolation_steps)):
    new_latent = slerp(float(t), encoding_2, encoding_3)
    tensor_list.append(new_latent)

images = []
print(range(interpolation_steps))
for batch in range(interpolation_steps):
    print(batch)
    images.append(
      model.generate_image(
            tensor_list[batch],
            negative_prompt=negative,
            batch_size=batch_size,
            diffusion_noise=noise,
                           )
                  )

In [ ]:
counter = 256
save_images(images,counter)

100%|██████████| 256/256 [03:45<00:00,  1.14it/s]


512

In [ ]:
# employ slerp
# divide points between 0 and 1 in linerspace based on the number of steps you want
tensor_list = []
for _, t in enumerate(np.linspace(0, 1, interpolation_steps)):
    new_latent = slerp(float(t), encoding_3, encoding_1)
    tensor_list.append(new_latent)

images = []
print(range(interpolation_steps))
for batch in range(interpolation_steps):
    print(batch)
    images.append(
      model.generate_image(
            tensor_list[batch],
            negative_prompt=negative,
            batch_size=batch_size,
            diffusion_noise=noise,
                           )
                  )

In [ ]:
counter = 512
save_images(images, counter)

In [ ]:
# employ slerp
# divide points between 0 and 1 in linerspace based on the number of steps you want
tensor_list = []
for _, t in enumerate(np.linspace(0, 1, interpolation_steps)):
    new_latent = slerp(float(t), encoding_4, encoding_5)
    tensor_list.append(new_latent)

images = []
print(range(interpolation_steps))
for batch in range(interpolation_steps):
    print(batch)
    images.append(
      model.generate_image(
            tensor_list[batch],
            negative_prompt=negative,
            batch_size=batch_size,
            diffusion_noise=noise,
                           )
                  )

In [ ]:
counter = 768
save_images(images, counter)

In [ ]:
plt.clf()
plt.close()
gc.collect()

# Circular Noise Walks

In [ ]:
# set noise seed for repeatability
seed = 44

# calculate noise input dimensions based on image dimensions
noise = tf.random.normal((768 // 8, 512// 8, 4), seed=seed)

circular_prompt = "galaxies in space from a distance, fractal, psytrance, high quality, masterpeice, high definition, highly detailed, elegant, sharp focus, digital painting, scifi, fantasy, center frame"
negative = "extra limbs, bad art, watermark, face, dull, pencils, error, malformed, low detail, jpeg artifacts, cropped, plain background, ugly, low-res, poorly drawn face, out of frame, poorly drawn hands, blurry, bad art, extra hands, bad anatomy, amputee, missing limbs, amputated"
encoding = tf.squeeze(model.encode_text(circular_prompt))
walk_steps = 1024
batch_size = 1
batches = walk_steps // batch_size

walk_noise_x = tf.random.normal(noise.shape, dtype=tf.float64)
walk_noise_y = tf.random.normal(noise.shape, dtype=tf.float64)

walk_scale_x = tf.cos(tf.linspace(0, 2, walk_steps) * math.pi)
walk_scale_y = tf.sin(tf.linspace(0, 2, walk_steps) * math.pi)
noise_x = tf.tensordot(walk_scale_x, walk_noise_x, axes=0)
noise_y = tf.tensordot(walk_scale_y, walk_noise_y, axes=0)
noise = tf.add(noise_x, noise_y)
batched_noise = tf.split(noise, batches)


In [ ]:
images = []
for batch in range(batches):
    print(batch)
    images.append(
        model.generate_image(
            encoding,
            batch_size=batch_size,
            negative_prompt=negative,
            diffusion_noise=batched_noise[batch],
        )
    )

In [ ]:
counter = 0
save_images(images, counter)

# Unassign Cloud GPU

In [ ]:
from google.colab import runtime
runtime.unassign()